In [50]:
import csv

csv_file_path = "Three_EPS_Req.csv"

def load_csv(file_path):
    requirements = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            requirements.append(row)
    return requirements

requirements_doc = load_csv(csv_file_path)
requirements_doc

[{'id': '245112',
  'Primary Text': 'Power Supply',
  'Artifact Type': 'Heading',
  'Safety': '',
  'Verification_Criteria': '',
  'isHeading': 'TRUE'},
 {'id': '245113',
  'Primary Text': "The Steering ECU shall ensure reliable operation within the specified voltage range of the vehicle electrical system while maintaining efficient power consumption under varying load conditions.\nOperating Range:\n• The Steering ECU shall operate on a 12V or 48V vehicle electrical system, with the following considerations:\n    '- Nominal Voltage: 12V system.\n    '- Operating Voltage Range: 10V to 24V.\n• The ECU shall handle voltage fluctuations within this range without degradation in functionality or performance.",
  'Artifact Type': 'SYS Requirement',
  'Safety': 'ASIL A',
  'Verification_Criteria': 'The Steering ECU shall monitor its power supply and generate the following Diagnostic Trouble Codes (DTCs) when specific fault conditions occur: \no Trigger DTC_Voltage_Failure if the supply voltage

In [143]:
import re

clean_text = re.sub(r"[\n\s•'-]+", " ", requirements_doc[2]['Primary Text'])

print(clean_text)

The ECU shall have a maximum power consumption of 50W during peak load conditions (e.g., full steering assist torque at low vehicle speeds). During normal operation, power consumption shall not exceed 20W, optimizing energy usage under typical driving conditions.


In [1]:
import os

os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"

In [71]:
# ## CHROMADB CODE

# # from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
# # from agno.vectordb.chroma import ChromaDb
# # from agno.embedder.sentence_transformer import SentenceTransformerEmbedder

# pdf_knowledge_base = PDFKnowledgeBase(
#     path="EPS_Info.pdf",
#     # Table name: ai.pdf_documents
#     vector_db=ChromaDb(collection="ECU_Information", 
#                       embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2"),
#                       path = "vectordb_agno",
#                       persistent_client=True),
#     reader=PDFReader(chunk=True),
# )
# #pdf_knowledge_base.load(upsert=True, recreate=True)

In [178]:
## LANCEDB CODE

from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.knowledge.csv import CSVKnowledgeBase
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.embedder.sentence_transformer import SentenceTransformerEmbedder
from agno.document.chunking.recursive import RecursiveChunking

pdf_knowledege_base = PDFKnowledgeBase(
    path="EPS_Info.pdf",
    # Table name: ai.pdf_documents
    vector_db=LanceDb(table_name="Information",
                      uri="vectordv_lancedb/lancedb_pdf",
                    embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2"),
                    search_type=SearchType.hybrid),
    reader=PDFReader(chunk=True, chunking_strategy=RecursiveChunking()),
)
# pdf_knowledege_base.load()

csv_knowledge_base = CSVKnowledgeBase(
    path="Three_EPS_Req.csv",
    # Table name: ai.csv_documents
     vector_db=LanceDb(table_name="Requirements",
                      uri="vectordv_lancedb/lancedb_csv",
                    embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2")
                )
)
# csv_knowledge_base.load()

In [181]:
from agno.knowledge.combined import CombinedKnowledgeBase

knowledge_base = CombinedKnowledgeBase(
    sources=[
        pdf_knowledege_base,
        csv_knowledge_base,
    ],
   vector_db=LanceDb(table_name="Combined_documents",
                      uri="vectordv_lancedb/combined_db",
                    embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2")
                ),
)

knowledge_base.load()

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: EPS_Info

INFO     Added 12 documents to knowledge base

INFO     Reading: Three_EPS_Req.csv

INFO     Added 1 documents to knowledge base

In [212]:
instruction = """Provide the response in below format. 
<understanding>
Explain your understanding and ask questions / clarification required for understanding complete requirement, if you have any doubts don't do assumptions.

</understanding>

<test_sceanrios>
Test scenarios to validate the requirement.
</test_scenarios>
"""

In [208]:
task=""" For 245114 requirement, perform the below tasks.
- First understand the requirement completely based on provided knowledge base and Explain your understanding and thoughts. If you don't understand the requirement or if you found any ambiguity or need clarification please ask questions.
- After understandint the requirements generate the test sceanarios to validate the requirement completely.

Note: Generate test scenarios for provided requirement ID only. Don't generate if the respective requirment ID is not provided."""

In [213]:
from agno.agent import Agent, RunResponse
from agno.models.groq import Groq

agent = Agent(
    knowledge=knowledge_base,
    search_knowledge=True,
    add_references=True,
    model=Groq(id="deepseek-r1-distill-llama-70b", temperature=0),
    description = "You are an AI assisant who will help understanding the requirements of automotive Electronic Circuit Units (ECU) and generate test scenarios to validate them",
    instructions = instruction,
    markdown=True,
    structured_outputs=True,
)

# agent.knowledge.load(recreate=False)

In [214]:
result = agent.run(task)
result

RunResponse(content="<understanding>\nThe requirement ID 245114 pertains to the power consumption limits of the Steering ECU. Specifically, it states that the ECU must have a maximum power consumption of 50W during peak load conditions (e.g., full steering assist torque at low vehicle speeds) and no more than 20W during normal operation. Additionally, it requires the ECU to trigger a DTC_OverCurrent if power consumption exceeds 50W for more than 1 second under normal conditions. The DTC should clear once the fault is resolved.\n\nTo ensure complete understanding, I have a few clarifying questions:\n\n1. Are there specific operating conditions or scenarios that should be prioritized for testing beyond what is explicitly mentioned?\n2. Should the test scenarios also verify the ECU's behavior when transitioning between peak and normal load conditions?\n3. Are there any additional fault conditions or edge cases related to power consumption that need to be addressed?\n\n</understanding>\n\n

In [215]:
print(result.content.strip())

<understanding>
The requirement ID 245114 pertains to the power consumption limits of the Steering ECU. Specifically, it states that the ECU must have a maximum power consumption of 50W during peak load conditions (e.g., full steering assist torque at low vehicle speeds) and no more than 20W during normal operation. Additionally, it requires the ECU to trigger a DTC_OverCurrent if power consumption exceeds 50W for more than 1 second under normal conditions. The DTC should clear once the fault is resolved.

To ensure complete understanding, I have a few clarifying questions:

1. Are there specific operating conditions or scenarios that should be prioritized for testing beyond what is explicitly mentioned?
2. Should the test scenarios also verify the ECU's behavior when transitioning between peak and normal load conditions?
3. Are there any additional fault conditions or edge cases related to power consumption that need to be addressed?

</understanding>

<test_scenarios>
### Test Scenar